In [1]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
from random import sample
import os

import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

сделать матрицу

In [2]:
os.listdir('data')    

['Dewas_NDVI.csv',
 'Dewas_points.csv',
 'Kaithal_NDVI.csv',
 'Kaithal_points.csv',
 'Karnal_NDVI.csv',
 'Karnal_points.csv']

In [3]:
data_points = pd.read_csv('data/Karnal_points.csv')

In [4]:
data_ndvi = pd.read_csv('data/Karnal_NDVI.csv')

In [5]:
data = data_points.merge(data_ndvi, left_on='gfid', right_on='gfid')

In [6]:
data['month'] = data['date'].apply(pd.to_datetime).dt.month
data['day'] = data['date'].apply(pd.to_datetime).dt.day

In [7]:
data_ndvi.head()

,gfid,datenum,date,ndvi
0,54001,0,2020-10-20,0.737
1,54001,1,2020-10-21,0.727
2,54001,2,2020-10-22,0.714
3,54001,3,2020-10-23,0.697
4,54001,4,2020-10-24,0.676


In [8]:
i = 1
ids_not_full = []
for id_ in data_points['gfid'].values:
    shp = data_ndvi[data_ndvi['gfid'] == id_].shape[0]
    if shp <= 160:
        #print(id_, ' ', shp)
        i += 1
        ids_not_full += [id_]
#print(i)

In [9]:
data2 = data[~data['gfid'].isin(ids_not_full)]

In [10]:
gfids = data2['gfid'].unique()

In [11]:
from numpy.random import choice

In [12]:
import numpy as np

In [13]:
np.random.seed(42)

In [14]:
np.random.shuffle(gfids)

In [15]:
train_ids = gfids[:210]
test_ids = gfids[210:]

In [38]:
import json
indices = {'Karnal_train_ids':train_ids.tolist(),
 'Karnal_test_ids':test_ids.tolist()}
with open('Karnal_indices.json', 'w') as f:
    json.dump(indices, f)

In [16]:
data2.head()

,gfid,state,district,village,lon,lat,wheat,datenum,date,ndvi,month,day
0,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,0,2020-10-20,0.737,10,20
1,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,1,2020-10-21,0.727,10,21
2,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,2,2020-10-22,0.714,10,22
3,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,3,2020-10-23,0.697,10,23
4,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,4,2020-10-24,0.676,10,24


In [17]:
data2 = data2.dropna()

In [18]:
data2.shape

(53505, 12)

In [19]:
data2.shape

(53505, 12)

In [20]:
for i in (3, 5, 7, 9):
    data2[f'ndvi_{i}'] = data2['ndvi'].rolling(i).mean()

In [21]:
datasets_train = []
datasets_test = []
y_train = []
y_test = []
for _ in gfids:
    rolling_data = []
    rolling_data.append(data2[data2['gfid']==_]['ndvi'].iloc[4:])
    for i in range(2, 11):
        rolling_data.append(data2[data2['gfid']==_]['ndvi'].rolling(i).mean())
    data_1_rolling = pd.concat(rolling_data, axis=1)
    data_1_rolling = data_1_rolling.dropna()
    data_1_rolling.columns = ['ndvi'] + [f'ndvi_{i}' for i in range(2,11)]
    if _ in train_ids:
        datasets_train.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_train += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]
    else:
        datasets_test.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_test += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]

In [22]:
data_train = torch.stack([torch.Tensor(dataset) for dataset in datasets_train])
data_test = torch.stack([torch.Tensor(dataset) for dataset in datasets_test])

In [23]:
def calc_out(h_in, w_in, ker, pad=0, stride=1):
    h_out = (h_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    w_out = (w_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    return h_out, w_out

In [24]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes  #number of classes
        
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=30, kernel_size=2, stride=2, padding=1),  # (b x 30 x 6 x 6)
            nn.ReLU(),
            nn.Conv2d(30, 64, 2, padding=0, stride=1),  # (b x 64 x 5 x 5)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),  # (b x 64 x 4 x 4)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1, inplace=True),
            nn.Linear(in_features=(64 * 4 * 4), out_features=256),
            nn.ReLU(),
            #nn.Dropout(p=0.5, inplace=True),
            #nn.Linear(in_features=256, out_features=256),
            #nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes),
            nn.Sigmoid()
        )
        #self.init_bias()

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[0].bias, 1)
        nn.init.constant_(self.net[2].bias, 1)
        #nn.init.constant_(self.net[4].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = x.view(-1, 64 * 4 * 4)  # reduce the dimensions for linear layer input
        return self.classifier(x)

In [25]:
alexnet = AlexNet(num_classes=2)

In [26]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=alexnet.parameters(), lr=0.0001)#1, weight_decay=0.5)

In [27]:
torch.autograd.set_detect_anomaly(True)

In [28]:
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

In [29]:
X = data_train
y = y_train_tensors

In [30]:
n_epochs = 100  # or whatever
batch_size = 30  # or whatever

for epoch in range(n_epochs):

    # X is a torch Variable
    permutation = torch.randperm(X.size()[0])

    for i in range(0, X.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X[indices], y[indices]

        outputs = alexnet.forward(batch_x) #forward pass
        #optimizer.zero_grad() #caluclate the gradient, manually setting to 0

        # obtain the loss function
        loss = criterion(outputs, batch_y)
        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.68764
Epoch: 1, loss: 0.68002
Epoch: 2, loss: 0.66706
Epoch: 3, loss: 0.66474
Epoch: 4, loss: 0.63459
Epoch: 5, loss: 0.67130
Epoch: 6, loss: 0.69843
Epoch: 7, loss: 0.60335
Epoch: 8, loss: 0.65880
Epoch: 9, loss: 0.67332
Epoch: 10, loss: 0.69289
Epoch: 11, loss: 0.65328
Epoch: 12, loss: 0.63858
Epoch: 13, loss: 0.65802
Epoch: 14, loss: 0.58473
Epoch: 15, loss: 0.69700
Epoch: 16, loss: 0.54818
Epoch: 17, loss: 0.64417
Epoch: 18, loss: 0.57673
Epoch: 19, loss: 0.63045
Epoch: 20, loss: 0.65611
Epoch: 21, loss: 0.57766
Epoch: 22, loss: 0.66180
Epoch: 23, loss: 0.55005
Epoch: 24, loss: 0.67641
Epoch: 25, loss: 0.66162
Epoch: 26, loss: 0.65080
Epoch: 27, loss: 0.57474
Epoch: 28, loss: 0.65237
Epoch: 29, loss: 0.59092
Epoch: 30, loss: 0.55369
Epoch: 31, loss: 0.59799
Epoch: 32, loss: 0.60253
Epoch: 33, loss: 0.62969
Epoch: 34, loss: 0.63220
Epoch: 35, loss: 0.61305
Epoch: 36, loss: 0.57766
Epoch: 37, loss: 0.63832
Epoch: 38, loss: 0.61999
Epoch: 39, loss: 0.64186
Epoch: 40,

In [31]:
X_test_tensors = data_test

In [32]:
test_predict = alexnet(X_test_tensors)#forward pass 

In [33]:
test_predict

tensor([[0.8886, 0.0902],
        [0.0236, 0.9702],
        [0.1137, 0.8567],
        [0.0113, 0.9860],
        [0.0198, 0.9765],
        [0.0320, 0.9641],
        [0.6066, 0.3818],
        [0.0506, 0.9438],
        [0.0416, 0.9545],
        [0.0276, 0.9682],
        [0.9685, 0.0234],
        [0.1966, 0.7947],
        [0.3676, 0.6209],
        [0.1403, 0.8543],
        [0.7726, 0.2199],
        [0.0463, 0.9492],
        [0.0273, 0.9680],
        [0.7387, 0.2615],
        [0.9491, 0.0387],
        [0.8994, 0.0860],
        [0.0773, 0.9150],
        [0.7253, 0.2250],
        [0.0970, 0.8977],
        [0.1113, 0.8721],
        [0.0165, 0.9794],
        [0.0975, 0.8922],
        [0.0161, 0.9808],
        [0.0348, 0.9577],
        [0.9262, 0.0592],
        [0.3079, 0.6780],
        [0.0035, 0.9955],
        [0.2176, 0.7780],
        [0.0489, 0.9435],
        [0.2888, 0.6559],
        [0.0244, 0.9731],
        [0.0295, 0.9672],
        [0.7530, 0.2458],
        [0.0169, 0.9808],
        [0.0

In [34]:
test_predict.argmax(dim=1)

tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1])

In [35]:
# 100 эпох, батч 30
print(classification_report(y_test_tensors, torch.argmax(test_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.67      0.60      0.63        20
           1       0.80      0.84      0.82        38

    accuracy                           0.76        58
   macro avg       0.73      0.72      0.73        58
weighted avg       0.75      0.76      0.76        58



In [36]:
train_predict = alexnet(X)#forward pass 

In [37]:
# 100 эпох, батч 30
print(classification_report(y_train_tensors, torch.argmax(train_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.71      0.59      0.65        88
           1       0.74      0.83      0.78       122

    accuracy                           0.73       210
   macro avg       0.72      0.71      0.71       210
weighted avg       0.73      0.73      0.72       210

